# KAKENから研究課題XMLファイルをダウンロードするプログラム

## 事前準備

データを取得する年度を指定する。KAKENは1964年度からのデータが保存されている。

In [ ]:
startyear = 1964
endyear = 2019

1. CiNiiウェブAPIの利用登録を済ませておく。詳細は https://support.nii.ac.jp/ja/cinii/api/developer のとおり。登録が完了したら、appidがメールで通知される。
2. appidはconfig.iniに保存しておく（配置場所は、このプログラムでは、"../../settings/config.ini"としているが、どこに置いてもいい）
3. ローカルに、MySQLかMariaDBをインストールしておく
4. 実際にテーブルを保存する場所としてのデータベース名は、kakenとして作成する。
5. データベースのusernameとpasswordを、config.iniに保存しておく

#### config.iniのサンプル

[CiNii_web_api]

appid = ****

[mariadb]

username = ****

password = ****




In [ ]:
import configparser
import os
import re

import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm_notebook

config = configparser.ConfigParser()
config.read("../../settings/config.ini")
appid = config["CiNii_web_api"]["appid"]
ROOT_URL = 'https://kaken.nii.ac.jp/opensearch/?appid=' + appid

ファイル保存用のフォルダを作成する

In [ ]:
os.makedirs("xml", exist_ok=True)

## 関数の定義

年度ごとの件数を取得する関数

In [ ]:
def get_number_of_record(year):
    url = ROOT_URL + "&format=xhtml" + "&s1=" + str(year) + "&s2=" + str(year) + "&o1=1"
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html, "lxml")
    tag = soup.select("p.search-term-selected")
    p = str(tag[0])  # 数字＝件数
    pattern = r"([+-]?[0-9]+\,?[0-9]*)"
    numberlist = re.findall(pattern, p)
    cnt = str(numberlist[0])
    cnt = int(cnt.replace(",", ""))
    return cnt

## ダウンロードを実行する

In [ ]:
for year in tqdm_notebook(range(startyear, endyear + 1)):
    cnt = get_number_of_record(year)
    for start in tqdm_notebook(range(1, cnt, 500)):
        rw = 500
        if start + rw > cnt:
            end = cnt
        else:
            end = start + rw - 1

        url = (
            ROOT_URL
            + "&format=xml"
            + "&s1="
            + str(year)
            + "&s2="
            + str(year)
            + "&o1=1"
            + "&st="
            + str(start)
            + "&rw="
            + str(rw)
        )
        r = requests.get(url)
        if r.status_code == 200:
            filename = "xml/" + str(year) + "_" + str(start) + "-" + str(end) + ".xml"
            with open(filename, mode="w", encoding="utf-8") as f:
                f.write(r.text)

## おわり

xmlフォルダを確認し、ファイルが保存されていればOK！